In [1]:
import sys
sys.path.insert(0, '../src/')

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz

import torch

import utils
from net import start_experiments

In [2]:
_A_obs = load_npz('../data/datasets/citeseer.npz')

In [3]:
val_share = 0.1
test_share = 0.05
seed = 481516234

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=False)

train_graph = sp.csr_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1])))
assert (train_graph.toarray() == train_graph.toarray().T).all()

In [7]:
models = start_experiments(num_experiments=5,
                           experiment_root='../logs/citeseer/Ours',
                           train_graph=train_graph,
                           H=8,
                           optimizer=torch.optim.Adam,
                           optimizer_args={'lr': 0.1,
                                           'weight_decay': 1e-7},
                           invoke_every=5,
                           steps = 100,
                           Model=Net,
                           val_edges=(val_ones, val_zeros))


Experiment_0
Step:   5/100, Loss: 7.19562, Edge-Overlap: 0.003
Step:  10/100, Loss: 5.51841, Edge-Overlap: 0.036
Step:  15/100, Loss: 4.16149, Edge-Overlap: 0.148
Step:  20/100, Loss: 3.37081, Edge-Overlap: 0.255
Step:  25/100, Loss: 2.79959, Edge-Overlap: 0.396
Step:  30/100, Loss: 2.39667, Edge-Overlap: 0.502
Step:  35/100, Loss: 2.12890, Edge-Overlap: 0.607
Step:  40/100, Loss: 1.95288, Edge-Overlap: 0.665
Step:  45/100, Loss: 1.83192, Edge-Overlap: 0.725
Step:  50/100, Loss: 1.74733, Edge-Overlap: 0.751
Step:  55/100, Loss: 1.68720, Edge-Overlap: 0.786
Step:  60/100, Loss: 1.64302, Edge-Overlap: 0.813
Step:  65/100, Loss: 1.60966, Edge-Overlap: 0.836
Step:  70/100, Loss: 1.58324, Edge-Overlap: 0.829
Step:  75/100, Loss: 1.56194, Edge-Overlap: 0.842
Step:  80/100, Loss: 1.54457, Edge-Overlap: 0.868
Step:  85/100, Loss: 1.52986, Edge-Overlap: 0.870
Step:  90/100, Loss: 1.51738, Edge-Overlap: 0.881
Step:  95/100, Loss: 1.50686, Edge-Overlap: 0.889
Step: 100/100, Loss: 1.49775, Edge-O